<a href="https://colab.research.google.com/github/Paolino1994/IntroIA/blob/main/Clase%203/Guia%20e%20Integrador%20Clase%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [282]:
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from functools import partial
from sklearn.decomposition import PCA

Ejecicio #1: Normalización

Muchos algoritmos de Machine Learning necesitan datos de entrada centrados y normalizados. Una normalización habitual es el z-score, que implica restarle la media y dividir por el desvío a cada feature de mi dataset.

In [283]:
def zScore(X):
  #X=np.array(X)
  mean=X.mean(axis=0)
  std=X.std(axis=0)
  return (X-mean)/std

In [284]:
X=np.array([1,80,3,55]).reshape(2,2)
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
if(np.allclose(zScore(X),X_scaled)):
  print("Scaler is working Great")


Scaler is working Great


Ejecicio #2: Remover filas y columnas con NaNs en un dataset

Dado un dataset, hacer una función que, utilizando numpy, filtre las columnas y las filas que tienen NaNs.

In [285]:
def removeNANs(X):
  nonNANRows=~np.isnan(X).any(axis=0)
  X=X.T[nonNANRows].T
  nonNANColumns=~np.isnan(X).any(axis=1)
  X=X[nonNANColumns]
  return X

In [286]:
X=np.array([0,1,2,3,4,5,np.NaN,7,8]).reshape(3,3)
print(X)
removeNANs(X)

[[ 0.  1.  2.]
 [ 3.  4.  5.]
 [nan  7.  8.]]


array([[1., 2.],
       [4., 5.],
       [7., 8.]])

Ejecicio #3: Reemplazar NaNs por la media de la columna

Dado un dataset, hacer una función que utilizando numpy reemplace los NaNs por la media de la columna.

In [287]:
def replaceNANs(X):
  means=np.nanmean(X, axis=0)
  Nans = np.where(np.isnan(X))
  X[Nans] = np.take(means, Nans[1]) 
  return X


In [288]:
X=np.array([0,1,2,3,4,5,np.NaN,7,8]).reshape(3,3)
replaceNANs(X)

array([[0. , 1. , 2. ],
       [3. , 4. , 5. ],
       [1.5, 7. , 8. ]])

Ejecicio #4: Dado un dataset X separarlo en 70 / 20 / 10

Como vimos en el ejercicio integrador, en problemas de Machine Learning es fundamental que separemos los datasets de n muestras, en 3 datasets de la siguiente manera:

    Training dataset: los datos que utilizaremos para entrenar nuestros modelos. Ej: 70% de las muestras.
    Validation dataset: los datos que usamos para calcular métricas y ajustar los hiperparámetros de nuestros modelos. Ej: 20% de las muestras.
    Testing dataset: una vez que entrenamos los modelos y encontramos los hiperparámetros óptimos de los mísmos, el testing dataset se lo utiliza para computar las métricas finales de nuestros modelos y analizar cómo se comporta respecto a la generalización. Ej: 10% de las muestras.

A partir de utilizar np.random.permutation, hacer un método que dado un dataset, devuelva los 3 datasets como nuevos numpy arrays.

In [371]:
def splitData(X,trainP=0.7,valP=0.2):
  if(trainP+valP>1):
    raise Exception('Sum is not %100')
  Xrand=np.random.permutation(X)
  trainQ=int(len(Xrand)*trainP)
  valQ=int(len(Xrand)*valP)+trainQ
  if(trainP+valP<1):
    testQ=len(Xrand)
    return Xrand[:trainQ],Xrand[trainQ:valQ],Xrand[valQ:testQ]
  else:
    return Xrand[:trainQ],Xrand[trainQ:]

In [372]:
X=np.array(range(0,300)).reshape(100,3)
for i in splitData(X):
  print(i.shape)

(70, 3)
(20, 3)
(10, 3)


INTEGRADOR

1.	Cargar los datos con objeto de clase Data (implementada por ustedes) con un método que cumpla esa función al pasarle la ruta. Hacer un split de los datos en train/test (usar 80/20)
2.	Tratar los nans con al menos dos de las técnicas vistas en clase. (pasarían a tener dos datasets para comparar en lo que sigue)
3.	Utilizar PCA para quedarse con las 3 CP.  (de cada uno del punto 2, idealmente usen su implementación, pero pueden usar las librerías)
4.	Crear una clase métrica base y una clase MSE que herede es ella. (esto viene de ejercicios anteriores)
5.	Crear una clase modelo base y clase regresión lineal que herede de ella.  (esto viene de ejercicios anteirores)
6.	Entrenar la regresión lineal sobre train. Calcular MSE sobre validation. (para todas las variantes que hayan hecho en 2) y comparar.


In [373]:
class AbstractMetric:
    def __init__(self, **kwargs):
        self.parameters = kwargs

    def __call__(self, *args, **kwargs):
        pass

def MSE(y,yHat):
  return np.sum((y-yHat)**2)/len(y)

class MSEClass (AbstractMetric):
    def __init__(self,**kwargs):
        AbstractMetric.__init__(self,**kwargs)
    def __call__(self):
        return MSE(self.parameters["y"],self.parameters["yHat"])

def splitXY(X):
  lenX=len(X)-1
  return X[:,0:lenX],X[:,-1]

In [424]:
class AbstractModel:
  def __init__(self, **kwargs):
        self.parameters = kwargs

  def __call__(self, *args, **kwargs):
        pass

def predict(a,b,X):
  return a + np.sum(b * X,axis=1)

def LinRegFunc(x_train,y_train):
  # Iniciar parámetros a y b -> Para hacer con numpy
  #np.random.seed(42)
  np.random.seed(1234)
  a = np.random.randn(1)
  b = np.random.randn(1)
  #print(a.shape,b.shape,x_train.shape)
  lr = 0.000001  # constante de aprendizaje
  n_epochs = 100000  # número de iteraciones
  for epoch in range(n_epochs):
      # Computar predicción del modelo
      yhat =  np.sum(b* x_train,axis=1)
      yhat =  a + yhat
      # Calcular error cuadrático medio (ECM)
      error = (y_train - yhat)
      loss = MSEClass(y=y_train,yHat=yhat)
      loss = loss()

      # Computar gradiente
      a_grad = -2 * error.mean()
      b_grad = -2 * np.dot(error,x_train  ).mean()
      # Actualizar parámetros
      a = a - (lr * a_grad)
      b = b - (lr * b_grad)
      #print(a,b)
  # Devolver predicción final

  return partial(predict, a, b)
# Calcular ECM final
class LinReg (AbstractModel):
    def __init__(self,**kwargs):
        AbstractMetric.__init__(self,**kwargs)
    def __call__(self):
        return LinRegFunc(self.parameters["x"],self.parameters["y"])

In [425]:
class Data:
  def __init__(self,path):
    self.data = np.genfromtxt(path, delimiter=';')
  
  def removeNANs(self):
    self.data = removeNANs(self.data)
  
  def replaceNANs(self):
    self.data = replaceNANs(self.data)

  def splitData(self):
    train,test=splitData(self.data,0.8,0.2)
    x_train,y_train=splitXY(train)
    x_test,y_test=splitXY(test)
    return x_train,y_train,x_test,y_test

In [426]:
#Load Data
#DAN -> Data with NaNs replaces by Average
#DNN -> Data with columns eliminated if had NaNs

path="clase3v2.csv"

myDataNoNans=Data(path)
myDataNoNans.removeNANs()
myDataAvgNans=Data(path)
myDataAvgNans.replaceNANs()

#Separate
DNN_trainX,DNN_trainY,DNN_valX,DNN_valY=myDataNoNans.splitData()
DAN_trainX,DAN_trainY,DAN_valX,DAN_valY=myDataAvgNans.splitData()

#PCA for DNN
pca = PCA(0.95)
pca.fit(DNN_trainX)

DNN_trainX_PCA = pca.transform(DNN_trainX)
DNN_valX_PCA = pca.transform(DNN_valX)
print(f"For DNN Reduced dimensions from {DNN_trainX.shape[1]} to {DNN_trainX_PCA.shape[1]}")

#PCA for DAN

pca = PCA(0.95)
pca.fit(DAN_trainX)

DAN_trainX_PCA = pca.transform(DAN_trainX)
DAN_valX_PCA = pca.transform(DAN_valX)
print(f"For DAN Reduced dimensions from {DAN_trainX.shape[1]} to {DAN_trainX_PCA.shape[1]}")

For DNN Reduced dimensions from 5 to 2
For DAN Reduced dimensions from 7 to 2


In [427]:
#Predict using No NANs
predictorOriginal=LinReg(x=DNN_trainX,y=DNN_trainY)()
yHatOriginal=predictorOriginal(DNN_valX)
errorOriginal=MSEClass(y=DNN_valY,yHat=yHatOriginal)

predictorPCA=LinReg(x=DNN_trainX_PCA,y=DNN_trainY)()
yHatPCA=predictorPCA(DNN_valX_PCA)
errorPCA=MSEClass(y=DNN_valY,yHat=yHatPCA)

print(f"Error using DNN on the original Dataset is {errorOriginal()}")
print(f"Error using DNN on the PCa Dataset is {errorPCA()}")

Error using DNN on the original Dataset is 17.703632973067545
Error using DNN on the PCa Dataset is 58.47953794438357


In [428]:
#Predict using AVG NANs
predictorOriginal=LinReg(x=DAN_trainX,y=DAN_trainY)()
yHatOriginal=predictorOriginal(DAN_valX)
errorOriginal=MSEClass(y=DAN_valY,yHat=yHatOriginal)

predictorPCA=LinReg(x=DAN_trainX_PCA,y=DAN_trainY)()
yHatPCA=predictorPCA(DAN_valX_PCA)
errorPCA=MSEClass(y=DAN_valY,yHat=yHatPCA)

print(f"Error using DAN on the original Dataset is {errorOriginal()}")
print(f"Error using DAN on the PCa Dataset is {errorPCA()}")

Error using DAN on the original Dataset is 27.034226256089084
Error using DAN on the PCa Dataset is 41.18685608402905
